# AdaIN-VC demo
This is a demonstration of AdaIN-VC that should work out of the box and be fairly quick to setup.

## Code Setup

In [1]:
!git clone https://github.com/yiftachbeer/AdaIN-VC
%cd AdaIN-VC

Cloning into 'AdaIN-VC'...
remote: Enumerating objects: 1117, done.
remote: Counting objects: 100% (1117/1117), done.
remote: Compressing objects: 100% (371/371), done.
remote: Total 1117 (delta 769), reused 1090 (delta 744), pack-reused 0
Receiving objects: 100% (1117/1117), 35.46 MiB | 26.80 MiB/s, done.
Resolving deltas: 100% (769/769), done.
/content/AdaIN-VC


In [2]:
%%capture

!python -m pip install -r requirements.txt

## Data Setup

We download a custom, smaller version of VCTK (all utterances of 5 speakers out of 110).

In [3]:
%%capture

!wget https://www.cs.huji.ac.il/~yiftach/VCTKmini.zip
!unzip VCTKmini.zip && rm VCTKmini.zip

In [4]:
%run adain-vc.py preprocess VCTKmini/wav48_silence_trimmed VCTKmini_mel

100% 5/5 [00:38<00:00,  7.77s/it]


## Training
The `n_steps` parameter can be adjusted depending on how long you want to wait. 

In [5]:
%run adain-vc.py train config.yaml VCTKmini_mel saved_models --n_steps 1000 --save_steps 100

Using speakers ['p229'] for validation.
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 3
wandb: You chose 'Don't visualize my results'
2021-06-11 16:54:07.029807: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
100% 1000/1000 [01:12<00:00, 13.71it/s, rec_los

## Inference

In [6]:
from IPython.display import Audio

We use the first sample for content, and the second for speaker:

In [7]:
Audio('VCTKmini/p226/p226_002_mic2.flac')

In [8]:
Audio('VCTKmini/p225/p225_003_mic2.flac')

We demonstrate the quality of the pretrained model along with the one we just trained:

In [9]:
%run adain-vc.py inference VCTKmini/p226/p226_002_mic2.flac VCTKmini/p225/p225_003_mic2.flac cvrt-trained.wav --model_path saved_models/model-1000.ckpt
%run adain-vc.py inference VCTKmini/p226/p226_002_mic2.flac VCTKmini/p225/p225_003_mic2.flac cvrt-pretrained.wav

inference.py:16: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:915.)
  wav = vocoder.generate([cvt.squeeze(0).data.T])
inference.py:16: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:915.)
  wav = vocoder.generate([cvt.squeeze(0).data.T])


In [10]:
Audio('cvrt-trained.wav')

In [11]:
Audio('cvrt-pretrained.wav')